# Final interface for now
Can have nodes to clean or just walls 


__user interface__ attempt #2 (not in play)
 - ask for size of maze (What width would u like? What height would u like?)
 - ask for difficulty (number of unclean squares)
 - generate a maze with demensions
 - start playing

In [3]:
def loadMap(name):
    arr = []
    with open(name, 'r') as reader:
        # Read and print the entire file line by line
        line = reader.readline()
        while line != '':  # The EOF char is an empty string
            arr.append(list(line.split(",")))
            line = reader.readline()
        
    for row in arr:
        del row[-1]
    for y in range(len(arr)):
        for x in range(len(arr[y])):
            arr[y][x] = int(arr[y][x])
    return arr


In [8]:
import random

room = [[0,1,0,1,1,0,1,0],
        [0,0,1,1,1,0,1,0],
        [1,0,0,1,0,0,1,1],
        [1,1,0,0,1,0,1,0]]

room = loadMap('Mazes/RoomMap.txt')
winW = 1200
winH = 600
dirtyChance = 0.00
showPath = True
showBestPath = True
CLEAN = 0
NOT_CLEAN = -1
WALL = 1
START = 2
FINISH = 3
xpos = 1
ypos = 1
finish = ()
start = ()
facing = "right"
map = []
playerPath = []
bestPath = []

def turnAround():
    global facing
    if facing == "right":
        facing = "left"
    elif facing == "left":
        facing = "right"
    elif facing == "up":
        facing = "down"
    elif facing == "down":
        facing = "up"
        
def turnRight():
    global facing
    if facing == "right":
        facing = "down"
    elif facing == "left":
        facing = "up"
    elif facing == "up":
        facing = "right"
    elif facing == "down":
        facing = "left"
        
def turnLeft():
    global facing
    if facing == "right":
        facing = "up"
    elif facing == "left":
        facing = "down"
    elif facing == "up":
        facing = "left"
    elif facing == "down":
        facing = "right"
    


def move(win,r,times):
    if times == 0:
        return
    global xpos,ypos
    if canMove():
        box_margin = 20
        w = winW/len(room[0])
        h = winH/len(room)
        if facing == "right":
            r.move(w*times,0)
            xpos += 1
        elif facing == "left":
            r.move(-w*times,0)
            xpos -= 1
        elif facing == "up":
            r.move(0,-h*times)
            ypos -= 1
        elif facing == "down":
            r.move(0,h*times)
            ypos += 1
        move(win,r,times-1)
    else:
        return False
        
def face(directionToFace):
    global facing
    facing = directionToFace

def canMove():
    if facing == "left" and xpos == 0 or facing == "left" and room[ypos][xpos-1] == WALL:
        return False
    elif facing == "right" and xpos == len(room[ypos])-1 or facing == "right" and room[ypos][xpos+1] == WALL:
        return False
    elif facing == "up" and ypos == 0 or facing == "up" and room[ypos-1][xpos] == WALL:
        return False
    elif facing == "down" and ypos == len(room)-1 or facing == "down" and room[ypos+1][xpos] == WALL:
        return False
    else:
        return True

def isClean():
    return room[ypos][xpos] == CLEAN

def placeRobot():
    global xpos, ypos
    for y in range(len(room)):
        for x in range(len(room[y])):
            if room[y][x] == START:
                xpos = x
                ypos = y

def addDirtySpots(chance = 0.01):
    global room
    for y in range(len(room)):
        for x in range(len(room[y])):
            if room[y][x] == NOT_CLEAN:
                room[y][x] = CLEAN
            if random.random() < chance and room[y][x] == CLEAN:
                room[y][x] = NOT_CLEAN
                
    
def moveToStart(win):
    global xpos, ypos
    xpos = start[0]
    ypos = start[1]
    w = winW/len(room[0])
    h = winH/len(room)
    bot = Circle(Point(xpos*w + w/2,ypos*h + h/2),5)
    bot.setFill("red")
    bot.draw(win)
    return bot
    
def printRoom():
    for y in range(len(room)):
        row = ""
        for x in range(len(room[y])):
            if x == xpos and y == ypos:
                row += " R "
            else:
                row += " " + str(room[y][x]) + " "
        print(row)
    print("Robot (" + str(xpos) + "," + str(ypos) + ") Facing: " + facing)
    print()

        
def cleanRoom(win):
    global room
    if not isClean():
        room[ypos][xpos] = CLEAN
        # map
        # w = winW/len(room[0])
        # h = winH/len(room)
        # rx = xpos*w
        # ry = ypos*h
        # r = Rectangle(Point(rx,ry), Point(rx+w,ry+h))
        # r.setFill("white")
        # r.draw(win)
    else:
        print("already clean")
                
def cleanedRoom():
    for row in room:
        for box in row:
            if box == NOT_CLEAN:
                return False
    return True
    
# display the win screen
def winDisplay(win):
    for item in win.items: # undraw all old items
            item.undraw()
    r = Rectangle(Point(0,0), Point(winW,winH))
    r.setFill("light green")
    t = Text(Point(winW/2,winH/2-60), "You won, all rooms are clean")
    t2 = Text(Point(winW/2,winH/2), "Good Job!")
    r.draw(win)
    t2.draw(win)
    t.draw(win)
       
# white for every clean location, gray for dirty location, trangle for robot
def display(win):
    global finish, start, map
    for item in win.items: # undraw all old items
        item.undraw()
    w = winW/len(room[0])
    h = winH/len(room)
    for y in range(len(room)):
        mapRow = []
        for x in range(len(room[y])):
            rx = x*w
            ry = y*h
            r = Rectangle(Point(rx,ry), Point(rx+w,ry+h))
            if room[y][x] == CLEAN:
                r.setFill("white")
            elif room[y][x] == NOT_CLEAN:
                r.setFill("grey")
            elif room[y][x] == WALL:
                r.setFill("black")
            elif room[y][x] == START:
                r.setFill("light green")
                start = (x,y)
            elif room[y][x] == FINISH:
                r.setFill("red2")
                finish = (x,y)
            mapRow.append(r)
            r.draw(win)
        map.append(mapRow)
    #draw robot
    # box_margin = 20
    # if facing == "right":
    #     p1 = Point(xpos*w+box_margin, ypos*h+box_margin)
    #     p2 = Point(xpos*w+box_margin, ypos*h+h-box_margin)
    #     p3 = Point(xpos*w+w-box_margin, ypos*h+(0.5*h))
    # elif facing == "left":
    #     p1 = Point(xpos*w+box_margin, ypos*h+(0.5*h))
    #     p2 = Point(xpos*w+w-box_margin, ypos*h+box_margin)
    #     p3 = Point(xpos*w+w-box_margin, ypos*h+h-box_margin)
    # elif facing == "up":
    #     p1 = Point(xpos*w+(0.5*w), ypos*h+box_margin)
    #     p2 = Point(xpos*w+box_margin, ypos*h+(0.5*h))
    #     p3 = Point(xpos*w+w-box_margin, ypos*h+(0.5*h))
    # elif facing == "down":
    #     p1 = Point(xpos*w+(0.5*w), ypos*h+h-box_margin)
    #     p2 = Point(xpos*w+box_margin, ypos*h+(0.5*h))
    #     p3 = Point(xpos*w+w-box_margin, ypos*h+(0.5*h))
        
    # tri = Polygon([p1,p2,p3])
    # tri.setFill("red")
    # tri.draw(win)
    bot = Circle(Point(xpos*w + w/2,ypos*h + h/2),5)
    bot.setFill("red")
    bot.draw(win)
    return bot
    
def moveBot(r,dist):
    if canMove():
        box_margin = 20
        w = winW/len(room[0])
        h = winH/len(room)
        if facing == "right":
            r.move(w*dist,0)
        elif facing == "left":
            r.move(-w*dist,0)
        elif facing == "up":
            r.move(0,-h*dist)
        elif facing == "down":
            r.move(0,h*dist)

        
def inputRoom():
    global room
    room = []
    rows =  input("How many rows do you want?")
    cols =  input("How many columns do you want?")
    if rows.isnumeric() and cols.isnumeric() and int(rows) > 1 and  int(cols) > 1:
        cols = int(cols)
        rows = int(rows)
        for y in range(rows):
            row = []
            for x in range(cols):
                row.append(random.randint(-1,1))
            room.append(row)
    else:
        print("Please enter numbers greater than 1.")
        inputRoom()

from graphics import *
def graphicsMain():
    global facing
    win = GraphWin("2D Robot", winW, winH, False)

    addDirtySpots(dirtyChance)
    placeRobot()
    print(room)
    robot = display(win)
    
    while True:
        if cleanedRoom() and finish == (xpos,ypos):
            update(30)
            winDisplay(win)
            
            if win.getMouse():
                win.close()
                break
                
        else:
            update(15)
            command = win.checkKey()
            if command == "q":
                break
            elif command == "Up":
                facing = "up"
            elif command == "Down":
                facing = "down"
            elif command == "Right":
                facing = "right"
            elif command == "Left":
                facing = "left"
            elif command == "space":
                move(win,robot,1)
            elif command == "c":
                cleanRoom(win)
                move(win,robot,0)
                robot = display(win)
            elif command == "r":
                robot.undraw()
                robot = moveToStart(win)
        
    

if __name__ == '__main__':
    graphicsMain()

[[0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1], [0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1], [0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1

GraphicsError: checkKey in closed window

In [ ]:
#quickest path
def quickestPath(maze):
    startPos = []
    finishPos = []
    
    path = [start] 
    
    
    
    
    path.append(finishPos)
    return path
    

# Finds the least moves for a robot to clean all the squares
No walls

In [12]:
room = [[0,1,0,1,1,0,1,0],
        [0,0,1,1,1,0,1,0],
        [1,0,0,1,0,0,1,1],
        [1,1,0,0,1,0,1,0]]
# room = loadMap('RoomMap2.txt')
CLEAN = 0
NOT_CLEAN = 1
xpos = 6
ypos = 4
facing = "right"

def move(times):
    if times == 0:
        return
    global xpos,ypos
    if canMove():
        if facing == "right":
            xpos += 1
        elif facing == "left":
            xpos -= 1
        elif facing == "up":
            ypos -= 1
        elif facing == "down":
            ypos += 1
        move(times-1)
    else:
        return False
        
def face(directionToFace):
    global facing
    facing = directionToFace

def canMove():
    if facing == "left" and xpos == 0:
        return False
    elif facing == "right" and xpos == len(room[ypos])-1:
        return False
    elif facing == "up" and ypos == 0:
        return False
    elif facing == "down" and ypos == len(room)-1:
        return False
    else:
        return True

def isClean():
    return room[ypos][xpos] == CLEAN
    
def printRoom():
    for y in range(len(room)):
        row = ""
        for x in range(len(room[y])):
            if x == xpos and y == ypos:
                row += " R "
            else:
                row += " " + str(room[y][x]) + " "
        print(row)
    print("Robot (" + str(xpos) + "," + str(ypos) + ")")
    print()

        
def cleanRoom():
    global room
    if not isClean():
        room[ypos][xpos] = CLEAN
    else:
        print("already clean")

def main():
    turns = 0
    while True: #find the closest dirty room - move to the room - clean the room - repeat until all rooms are clean
        closest = [-1,-1,len(room) * len(room[0])] # x pos, y pos, moves away
        for y in range(len(room)):
            for x in range(len(room[y])):
                if room[y][x] == 1: # if its a dirty room
                    newDist = abs(xpos-x) + abs(ypos-y) # calcualte the # of moves away from current pos
                    if newDist < closest[2]: # if the dist is closer then the previous closest
                        closest = [x,y,newDist] # make it the new closest
        if closest[0] == -1: # if no dirty rooms left
            break
        # find direction and move horizantally 
        if xpos < closest[0]:
            face("right")
        elif xpos > closest[0]:
            face("left")
        move(abs(xpos-closest[0]))
        # find direction and move vertically 
        if ypos < closest[1]:
            face("down")
        elif ypos > closest[1]:
            face("up")
        move(abs(ypos-closest[1]))
        # clean the room robot is now in
        cleanRoom()
        turns += 1
        printRoom() # print out what the room looks like 
        
    print(str(turns) + " moves until every square is clean")

if __name__ == '__main__':
    main()

 0  1  0  1  1  0  1  0 
 0  0  1  1  1  0  1  0 
 1  0  0  1  0  0  1  1 
 1  1  0  0  1  0  R  0 
Robot (6,3)

 0  1  0  1  1  0  1  0 
 0  0  1  1  1  0  1  0 
 1  0  0  1  0  0  R  1 
 1  1  0  0  1  0  0  0 
Robot (6,2)

 0  1  0  1  1  0  1  0 
 0  0  1  1  1  0  R  0 
 1  0  0  1  0  0  0  1 
 1  1  0  0  1  0  0  0 
Robot (6,1)

 0  1  0  1  1  0  R  0 
 0  0  1  1  1  0  0  0 
 1  0  0  1  0  0  0  1 
 1  1  0  0  1  0  0  0 
Robot (6,0)

 0  1  0  1  R  0  0  0 
 0  0  1  1  1  0  0  0 
 1  0  0  1  0  0  0  1 
 1  1  0  0  1  0  0  0 
Robot (4,0)

 0  1  0  R  0  0  0  0 
 0  0  1  1  1  0  0  0 
 1  0  0  1  0  0  0  1 
 1  1  0  0  1  0  0  0 
Robot (3,0)

 0  1  0  0  0  0  0  0 
 0  0  1  R  1  0  0  0 
 1  0  0  1  0  0  0  1 
 1  1  0  0  1  0  0  0 
Robot (3,1)

 0  1  0  0  0  0  0  0 
 0  0  R  0  1  0  0  0 
 1  0  0  1  0  0  0  1 
 1  1  0  0  1  0  0  0 
Robot (2,1)

 0  R  0  0  0  0  0  0 
 0  0  0  0  1  0  0  0 
 1  0  0  1  0  0  0  1 
 1  1  0  0  1  0  0 

In [2]:
%run Generation.ipynb


|F|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|#|.|#|.|.|.|.|.|.|.|#|.|#|.|.|#|.|.|#|.|.|#|.|#|.|#|.|.|#|.|.|
|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|#|.|.|.|.|.|.|#|.|.|.|.|#|.|#|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|#|.|#|.|.|#|.|.|
|#|#|#|#|#|#|#|#|#|#|#|#|#|#|#|.|#|#|#|#|#|#|#|#|#|.|#|#|#|#|.|#|.|.|#|.|#|.|.|#|.|#|.|.|#|.|.|#|.|.|#|.|.|.|.|.|.|#|.|.|
|.|.|.|.|.|.|.|.|.|.|#|.|.|.|.|.|.|.|.|.|.|.|#|.|.|#|.|.|.|#|.|#|#|.|#|#|#|#|#|.|#|#|#|#|#|#|#|#|#|.|#|#|#|#|#|#|#|#|.|.|
|#|#|#|#|.|#|#|.|#|#|#|#|#|.|#|#|#|#|#|#|.|#|#|.|.|.|.|.|.|#|.|#|.|.|#|.|.|.|.|.|.|.|.|.|#|.|.|.|.|.|.|.|.|.|.|.|.|#|.|.|
|.|.|.|.|.|.|.|.|.|.|#|.|.|.|.|.|.|.|.|#|.|.|#|.|#|#|#|.|#|#|.|.|.|.|#|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|#|.|.|
|.|#|#|#|.|#|.|#|#|.|.|#|.|#|#|#|#|.|#|#|.|.|#|.|.|#|.|.|.|#|#|#|.|#|#|#|.|#|#|#|#|#|#|#|#|#|#|#|#|#|#|#|#|#|#|#|#|#|.|.|
|.|#|.|.|.|#|.|#|.|.|#|.|.|.|.|.|#|.|.|.|.|.|.|.|.|.|.|.|.|#|.|#|.|.|#|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.|#|.|.|
|.|.|.|.|.|#|.|.|.|.|#|.

# First attempt (using arcade)

In [7]:
import arcade
import numpy as np
# %run Generation.ipynb

# Set constants for the screen size
SCREEN_WIDTH = 600
SCREEN_HEIGHT = 600

# maze = [
#     ["#", ".", ".", ".", ".", ".", ".", ".", ".", "#", "."],
#     [".", ".", "#", "#", "#", "#", "#", ".", ".", ".", "."],
#     [".", "#", "#", ".", ".", ".", ".", ".", "#", ".", "."],
#     [".", ".", "#", ".", ".", ".", ".", ".", "#", ".", "."],
#     ["#", ".", "#", ".", ".", ".", ".", ".", "#", ".", "#"],
#     [".", ".", "#", ".", ".", ".", ".", ".", ".", ".", "."],
#     [".", "#", "#", "#", "#", ".", ".", "#", ".", "#", "."],
#     [".", "#", ".", ".", "F", "#", ".", "#", "#", "#", "."],
#     [".", "#", ".", "#", "#", "#", "#", "#", ".", ".", "."],
#     [".", "#", ".", "#", ".", ".", ".", ".", ".", "#", "."],
#     [".", "#", ".", "#", "#", "#", "#", "#", ".", "#", "."],
#     [".", "#", ".", ".", ".", "#", ".", ".", ".", ".", "."],
#     [".", "#", "#", "#", ".", "#", ".", "#", "#", ".", "."],
#     ["S", "#", ".", ".", ".", ".", ".", ".", ".", "#", "."]
# ]


class MyGame(arcade.Window):
    """ Main application class. """

    def __init__(self, maze, width, height):
        super().__init__(width, height)
        
        self.width = width
        self.height = height
        self.maze = maze
        self.stats = maze.stats
        arcade.set_background_color(arcade.color.AMAZON)

    def setup(self):
        # Set up your game here
        self.state = "Menu" # Menu, Daily, Levels, Success, Fail, Credits
        
        self.padding = 10
        self.charW = (self.width - self.padding * 2) / self.maze.width
        self.charH = (self.height - self.padding * 2) / self.maze.height
        
        #Start placement
        for key in self.stats:
            if self.stats[key] == "S":
                self.player = key
                print((key[0] * self.charW + self.padding, key[1] * self.charH + self.padding))
                self.playerPos = (key[0] * self.charW + self.padding, key[1] * self.charH + self.padding)
                    
        pass

    def on_draw(self):
        playerPos = self.playerPos
        """ Render the screen. """
        arcade.start_render()
        # Your drawing code goes here
        
        #Draw border
        arcade.draw_lrtb_rectangle_filled(0, self.width, self.height, 0, arcade.color.GRAY)
        # Draw maze area
        arcade.draw_lrtb_rectangle_filled(self.padding, self.width - self.padding, self.height - self.padding, self.padding, arcade.color.WHITE)
        # Draw player
        arcade.draw_lrtb_rectangle_filled(playerPos[0], playerPos[0] + self.charW, playerPos[1] + self.charH, playerPos[1], arcade.color.GREEN)
        
        # Draw walls and finish
        for key in self.stats:
            xPos = key[0] * self.charW + self.padding
            yPos = key[1] * self.charH + self.padding
            if self.stats[key] == "#":
                arcade.draw_lrtb_rectangle_filled(xPos, xPos + self.charW, yPos + self.charH, yPos, arcade.color.BLACK)
            if self.stats[key] == "F":
                arcade.draw_lrtb_rectangle_filled(xPos, xPos + self.charW, yPos + self.charH, yPos, arcade.color.RED)
                    
        

    def update(self, delta_time):
        """ the game logic goes here. """
        pass
    
    def on_key_press(self, key, modifiers):
        """Called whenever a key is pressed. """
        if key == arcade.key.UP and self.player[1] + 1 < self.maze.height and self.stats[(self.player[0], self.player[1] + 1)] != "#":
            self.player = (self.player[0], self.player[1] + 1)
            self.playerPos = (self.playerPos[0], self.playerPos[1] + self.charH)
        elif key == arcade.key.DOWN and self.player[1] - 1 >= 0 and self.stats[(self.player[0], self.player[1] - 1)] != "#":
            self.player = (self.player[0], self.player[1] - 1)
            self.playerPos = (self.playerPos[0], self.playerPos[1] - self.charH)
        elif key == arcade.key.LEFT and self.player[0] - 1 >= 0 and self.stats[(self.player[0] - 1, self.player[1])] != "#":
            self.player = (self.player[0] - 1, self.player[1])
            self.playerPos = (self.playerPos[0] - self.charW, self.playerPos[1])
        elif key == arcade.key.RIGHT and self.player[0] + 1 < self.maze.width and self.stats[(self.player[0] + 1, self.player[1])] != "#":
            self.player = (self.player[0] + 1, self.player[1])
            self.playerPos = (self.playerPos[0] + self.charW, self.playerPos[1])


def main():
    game = MyGame(g1, SCREEN_WIDTH, SCREEN_HEIGHT)
    game.setup()
    arcade.run()


if __name__ == "__main__":
    main()


(207.2, 207.2)


KeyboardInterrupt: 

# 2nd Version - Built into generation file
Pretty slow but cool looking

In [4]:
g1 = graph(maze)
w = 60
h = 36
g1.primitive(w, h)
g1.optimize_maze()
print(g1.start)
g1.print_graph()
arr = dictToArr(g1.stats,w,h)
game1 = MazeLevel(g1)
game1.playLevel()

(38, 18)
|.|#|.|.|.|#|.|.|.|.|.|.|.|.|.|#|.|.|.|.|.|.|.|.|.|.|.|.|.|#|.|.|.|.|.|#|.|.|.|#|.|#|.|#|.|#|.|#|.|.|.|#|.|#|.|#|.|.|.|.|
|.|#|#|#|.|#|#|#|#|#|.|#|#|#|.|#|#|#|#|#|.|#|#|#|#|#|.|#|#|#|.|#|#|#|#|#|#|#|.|#|.|#|.|#|.|#|.|#|#|#|.|#|.|#|.|#|#|#|.|#|
|.|.|.|.|.|.|.|.|.|.|.|#|.|.|.|.|.|.|.|#|.|#|.|.|.|.|.|#|.|#|.|.|.|.|.|#|.|#|.|#|.|.|.|#|.|.|.|#|.|.|.|.|.|.|.|.|.|#|.|.|
|#|#|.|#|#|#|.|#|#|#|#|#|#|#|.|#|#|#|#|#|#|#|.|#|#|#|.|#|.|#|.|#|#|#|#|#|.|#|.|#|.|#|#|#|.|#|#|#|.|#|#|#|#|#|.|#|#|#|.|#|
|.|.|.|.|.|#|.|#|.|.|.|.|.|#|.|.|.|#|.|#|.|.|.|#|.|.|.|.|.|.|.|#|.|#|.|#|.|.|.|#|.|#|.|.|.|.|.|.|.|.|.|#|.|.|.|.|.|.|.|.|
|#|#|#|#|#|#|#|#|#|#|.|#|.|#|#|#|.|#|.|#|#|#|#|#|#|#|.|#|#|#|.|#|.|#|.|#|.|#|#|#|.|#|#|#|.|#|.|#|#|#|.|#|.|#|.|#|.|#|#|#|
|F|#|.|.|.|#|.|#|.|#|.|#|.|.|.|.|.|.|.|#|.|.|.|#|.|.|.|#|.|.|.|.|.|.|.|.|.|.|.|#|.|#|.|.|.|#|.|.|.|#|.|#|.|#|.|#|.|#|.|.|
|.|#|#|#|.|#|.|#|.|#|#|#|.|#|#|#|.|#|.|#|#|#|.|#|#|#|.|#|.|#|.|#|#|#|#|#|.|#|#|#|.|#|.|#|#|#|#|#|#|#|#|#|#|#|#|#|#|#|.|#|
|.|.|.|#|.|.|.|

Terminator: 